In [ ]:
#idk diference between cv and cv2. I think cv2 has class functionality
#import cv

In [1]:
#library needed on beaglebone
import cv2
import numpy as np
import time

In [2]:
import matplotlib
import matplotlib.pyplot as plt

In [47]:
def get_480p(file):
    img=cv2.imread(file)
    img=cv2.resize(img,(640,480))
    gray_img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img,gray_img
def get_cut_480p(file):
    img=cv2.imread(file)
    img1=img[320:,:]
    gray_img=cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    return img1,gray_img
def get_360p(file):
    img=cv2.imread(file)
    img=cv2.resize(img,(320,240))
    gray_img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img,gray_img
def display(img):
    try:
        b,g,r = cv2.split(img)
        img2 = cv2.merge([r,g,b])
        imgplot = plt.imshow(img)
        plt.show()
    except:    
        imgplot = plt.imshow(img,cmap='gray')
        plt.show()
def hough_trans(img,gray,can_min=50,can_max=350,vote=100,plot_canny=0):
    t1 = time.time()
    edges = cv2.Canny(gray,can_min,can_max,apertureSize = 3)
    display(edges)
    t2 = time.time()
    lines = cv2.HoughLines(edges,1,np.pi/180,125)
    t3 = time.time()
    print 'canny',t2 - t1
    print 'hough',t3 - t2
    #print lines
    print lines
    for rho,theta in lines[0]:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))
        cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)
    if plot_canny==1:
        display(edges)
    display(img)
def hough_transp(img,gray,can_min=50,can_max=350,vote=100,minLineLength = 100,maxLineGap = 20,plot_canny=0):
    #hough that uses less test points, less computations
    img = cv2.imread('4-knee.jpg')
    t1 = time.time()
    edges = cv2.Canny(gray,can_min,can_max,apertureSize = 3)
    t2 = time.time()
    lines = cv2.HoughLinesP(edges,1,np.pi/180,vote,minLineLength,maxLineGap)
    t3 = time.time()
    print 'canny',t2 - t1
    print 'hough',t3 - t2
    #plot lines
    for x1,y1,x2,y2 in lines[0]:
        cv2.line(img,(x1,y1),(x2,y2),(255,0,0),2)
    if plot_canny==1:
        display(edges)
    display(img)
def hough_trans_blur(img,gray,can_min=50,can_max=350,vote=100,plot_canny=0):
    t1 = time.time()
    blur = cv2.blur(gray,(2,2))
    t2 = time.time()
    edges = cv2.Canny(gray,can_min,can_max,apertureSize = 3)
    t3 = time.time()
    lines = cv2.HoughLines(edges,1,np.pi/180,vote)
    t4 = time.time()
    print 'blur', t2 - t1
    print 'canny',t3 - t2
    print 'hough',t4 - t3
    #print lines
    for rho,theta in lines[0]:
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + 1000*(-b))
        y1 = int(y0 + 1000*(a))
        x2 = int(x0 - 1000*(-b))
        y2 = int(y0 - 1000*(a))
        cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)
    if plot_canny==1:
        display(edges)
    display(img)

In [48]:
img1,gray=get_cut_480p('2-knee.jpg')
display(gray)
blur = cv2.bilateralFilter(gray,9,75,75)
display(blur)

In [49]:
hough_trans(img1,blur,vote=150,can_min=50,can_max=200)

canny 10.0288329124
hough 0.02481508255
[[[ 523.            1.62315619]
  [   3.            1.57079637]
  [  10.            1.57079637]
  [   7.            1.57079637]
  [-878.            2.60054064]
  [  -8.            1.58824956]
  [ -10.            1.58824956]
  [  17.            1.57079637]
  [ -32.            1.60570288]
  [  -4.            1.58824956]
  [  73.            1.53588974]
  [  39.            1.55334306]
  [ -35.            1.60570288]
  [ -12.            1.58824956]
  [  14.            1.57079637]
  [ -16.            1.58824956]
  [ -18.            1.58824956]
  [ -74.            1.6406095 ]
  [  12.            1.57079637]
  [  22.            1.57079637]
  [  72.            1.43116999]
  [  37.            1.55334306]
  [  61.            1.46607661]
  [ -37.            1.60570288]
  [-860.            2.58308721]
  [ -54.            1.62315619]
  [  70.            1.53588974]
  [ 539.            1.60570288]
  [ -40.            1.60570288]
  [  67.            1.53588974]


In [ ]:
#try only the area directly in front of the robot.

In [ ]:
#get low image resolution or take midsection of high res.
#blur, use the bit operator to only output where the sidewalk is, travel on right side.
#perhaps fill in bits of data. 
#in forest area use the high intensity light as a sign of the sidewalk being there.

In [ ]:
#try to use the averaging filters to get the whole concrete slab
img,gray=get_480p('8.jpg')
blur = cv2.blur(gray,(2,2))
display(blur)
hough_trans(img,gray)

In [ ]:
#use bilateral filtering to remove texture(concrete) but is cpu intensive. 
img,gray=get_480p('8.jpg')
blur = cv2.bilateralFilter(gray,9,75,75)
display(blur)
hough_trans(img,gray)

In [ ]:
#for tree area use just the green element, anything in between high amounts of green is sidewalk.
#use the binary operators to set up this mappying.
img,gray=get_480p('4-knee.jpg')
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
lower_blue = np.array([50,50,110])
upper_blue = np.array([255,255,130])
mask = cv2.inRange(hsv, lower_blue, upper_blue)
 # Bitwise-AND mask and original image
res = cv2.bitwise_and(img,img, mask= mask)
display(img)
display(mask)
display(res)

In [ ]:
cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
img=cv2.imread('4-knee.jpg')

In [ ]:
#returns (height, width, pixel element number)
height, width, pixel=img.shape
print img.shape

In [ ]:
imgplot = plt.imshow(img)
plt.show()

In [ ]:
img2 = img[280:340, 330:390]#take section of the picture
img[273:333, 100:160] = img2#put that section on original picutre

In [ ]:
imgplot = plt.imshow(img)
plt.show()

In [ ]:
b,g,r = cv2.split(img)#full splitting into colors
b = img[:,:,0]#individual blue color
imgplot = plt.imshow(b)
plt.show()

In [ ]:
#Grayscale
gray_img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
imgplot = plt.imshow(gray_img)
plt.show()

In [ ]:
#HSV
hsv_img=cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
imgplot = plt.imshow(hsv_img)
plt.show()

In [ ]:
print img[0,0]
print gray_img[0,0]
print hsv_img[0,0]

In [64]:
#capture video
cap = cv2.VideoCapture(0)
while 1:
    ret, frame = cap.read()
    if ret==True:#check if cap.read() returned frame
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        display(gray)
        #image recognition code here
    break

In [ ]:
#canny edge detection
img = cv2.imread('4-knee.jpg',0)
edges = cv2.Canny(img,100,200)

plt.subplot(121),plt.imshow(img,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])

plt.show()

In [ ]:
img = cv2.imread('1-a.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,160,300,apertureSize = 3)
lines = cv2.HoughLines(edges,1,np.pi/180,200)
#print lines
for rho,theta in lines[0]:
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))
    cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)

plt.subplot(121),plt.imshow(edges,cmap = 'gray')
plt.subplot(122),plt.imshow(img,cmap = 'gray')
#plt.imshow(img,cmap='gray')
plt.show()
#cv2.imwrite('houghlines1.jpg',img)

In [ ]:
#hough that uses less test points, less computations
img = cv2.imread('4-knee.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,50,150,apertureSize = 3)
minLineLength = 100
maxLineGap = 20
lines = cv2.HoughLinesP(edges,1,np.pi/180,100,minLineLength,maxLineGap)
#print lines
for x1,y1,x2,y2 in lines[0]:
    cv2.line(img,(x1,y1),(x2,y2),(255,0,0),2)
plt.subplot(121),plt.imshow(edges,cmap = 'gray')
plt.subplot(122),plt.imshow(img,cmap = 'gray')
plt.show()
#cv2.imwrite('houghlines2.jpg',img)